In [ ]:
## First attempt at creating a queuing simulation
## ATM jsut spits out an avg wait time, want to correlate this with the graphs on the other jupyter noteboko (which still need to be checked and verified)
## If that works, hopeuflly can combine al lhese in order ot produce a simulation for different variables and display graphics for each

import heapq
import random

class Event:
    def __init__(self, time, event_type, customer_id):
        self.time = time
        self.event_type = event_type  # "arrival" or "departure"
        self.customer_id = customer_id

    def __lt__(self, other):
        return self.time < other.time

class QueueSimulation:
    def __init__(self, arrival_rate, service_rate, num_servers, sim_time):
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.num_servers = num_servers
        self.sim_time = sim_time
        self.event_queue = []  # priority queue for events
        self.server_busy = 0  # # of busy servers
        self.wait_times = []  # record of individual wait times
        self.queue = []  # queue for customers waiting for service

    def generate_interarrival_time(self):
        return random.expovariate(self.arrival_rate)

    def generate_service_time(self):
        return random.expovariate(self.service_rate)

    def run(self):
        # schedule the first arrival
        customer_id = 0
        first_arrival = self.generate_interarrival_time()
        heapq.heappush(self.event_queue, Event(first_arrival, "arrival", customer_id))

        while self.event_queue:
            event = heapq.heappop(self.event_queue)

            # End simulation if  exceed sim_time
            if event.time > self.sim_time:
                break

            if event.event_type == "arrival":
                self.handle_arrival(event)
            elif event.event_type == "departure":
                self.handle_departure(event)

        # Calculate avg wait time
        avg_wait = sum(self.wait_times) / len(self.wait_times) if self.wait_times else 0
        return avg_wait

    def handle_arrival(self, event):
        customer_id = event.customer_id

        # schedule  next arrival
        next_arrival = event.time + self.generate_interarrival_time()
        heapq.heappush(self.event_queue, Event(next_arrival, "arrival", customer_id + 1))

        if self.server_busy < self.num_servers:
            # Server available, start service immediately
            self.server_busy += 1
            service_time = self.generate_service_time()
            departure_time = event.time + service_time
            heapq.heappush(self.event_queue, Event(departure_time, "departure", customer_id))
        else:
            # All servers busy, add to queue
            self.queue.append((event.time, customer_id))  # Store arrival time and customer ID

    def handle_departure(self, event):
        self.server_busy -= 1

        # check if customers waiting in the queue
        if self.queue:
            arrival_time, customer_id = self.queue.pop(0)  # FIFO
            wait_time = event.time - arrival_time  # Calculate waiting time
            self.wait_times.append(wait_time)

            # Start service for the next customer
            self.server_busy += 1
            service_time = self.generate_service_time()
            departure_time = event.time + service_time
            heapq.heappush(self.event_queue, Event(departure_time, "departure", customer_id))

# run the simulation with high arrival rate -- example
arrival_rate = 50  # Very high arrival rate
service_rate = 5    # Service rate per server
num_servers = 10    # Number of servers
sim_time = 600      # Total simulation time

sim = QueueSimulation(arrival_rate, service_rate, num_servers, sim_time)
avg_wait = sim.run()
print(f"Average Wait Time: {avg_wait:.2f}")
